In [2]:
# %load symmetric_encryption.py
#!/usr/bin/env/python3

import urllib.request
from Crypto.Cipher import AES
import binascii
import base64
import random
import os
import zlib

################################################################################
# CS 284 Padding Utility Functions
################################################################################

# s is a bytearray to pad, k is blocklength
# you won't need to change the block length
def cmsc284pad(s,k=16):
  if k > 255:
    print("pkcs7pad: padding block length must be less than 256")
    return bytearray()
  n = k - (len(s) % k)
  if n == 0:
    n = k
  for i in range(1,n+1):
    s.extend([i])
  return s

# s is bytes to pad, k is blocklength
# you won't need to change the block length
def cmsc284padbytes(s,k=16):
  if k > 255:
    raise Exception("pkcs7pad: padding block length must be less than 256")
  n = k - (len(s) % k)
  if n == 0:
    n = k
  for i in range(1,n+1):
    s += chr(i).encode("utf-8")
  return s

# s is bytes to unpad, k is blocklength
# you won't need to change the block length
def cmsc284unpad(s,k=16):
  if not cmsc284checkpadding(s,k):
    print("cmsc284unpad: invalid padding")
    return b''
  n = s[len(s)-1]
  return s[:len(s)-n]

# checks padding on s and returns a boolean
# you won't need to change the block length
def cmsc284checkpadding(s,k=16):
  if(len(s) == 0):
    #print("Invalid padding: String zero length"%k) 
    return False
  if(len(s)%k != 0): 
    #print("Invalid padding: String is not multiple of %d bytes"%k) 
    return False
  n = s[len(s)-1]
  if n > k or n == 0:
    return False
  else: 
    for i in range(n):
      if s[len(s)-1-i] != (n-i):
        return False
  return True

################################################################################
# Function for querying the server
################################################################################

PPS2SERVER = "http://cryptoclass.cs.uchicago.edu/"
def make_query(task, cnetid, query):
  DEBUG = False
  if DEBUG:
    print("making a query")
    print("Task:", task)
    print("CNET ID:", cnetid)
    print("Query:", query)
  if (type(query) is bytearray) or (type(query) is bytes):
    url = PPS2SERVER + urllib.parse.quote_plus(task) + "/" + urllib.parse.quote_plus(cnetid) + "/" + urllib.parse.quote_plus(base64.urlsafe_b64encode(query)) + "/"
  else:
    url = PPS2SERVER + urllib.parse.quote_plus(task) + "/" + urllib.parse.quote_plus(cnetid) + "/" + urllib.parse.quote_plus(base64.urlsafe_b64encode(query.encode('utf-8'))) + "/"
  if DEBUG:
    print("Querying:", url)

  with urllib.request.urlopen(url) as response:
    raw_answer = response.read()
    answer = base64.urlsafe_b64decode(raw_answer)
    if DEBUG:
      print("Answer:", answer)
    return answer
  return None


################################################################################
# Problem 1 SOLUTION
################################################################################

def problem1(cnetid):
  return b''


################################################################################
# Problem 2 SOLUTION
################################################################################

def problem2(cnetid):
  return b''


################################################################################
# Problem 3 SOLUTION
################################################################################

def problem3(cnetid):
  return b''


################################################################################
# Problem 4 SOLUTION
################################################################################

def problem4(cnetid):
  return b''


################################################################################
# Problem 5 SOLUTION
################################################################################

def problem5(cnetid):
  return b''

################################################################################
# Problem 6 SOLUTION
################################################################################

def problem6(cnetid):
  return b''

In [3]:
from collections import Counter

In [20]:
# get padded FLAG len
padded_flag_len = len(make_query('three', 'ruolinzheng', ''))

In [21]:
padded_flag_len

48

In [17]:
# determine unpadded FLAG len, [padded_flag_len - 16, padded_flag_len - 1]
len(make_query('three', 'ruolinzheng', bytes(14))), \
len(make_query('three', 'ruolinzheng', bytes(15)))

(48, 64)

In [25]:
# bytes(16) || FLAG is either padded_flag_len or padded_flag_len + 16
prev = -1
for i in range(1, 17):
  curr = len(make_query('three', 'ruolinzheng', bytes(i)))
  if prev == padded_flag_len and curr == padded_flag_len + 16:
    print('FLAG length', padded_flag_len - i)
    break
  prev = curr

FLAG length 33


In [26]:
# unpadded FLAG len 33
flag_len = 33

In [40]:
# manipulate query content to get 1 byte || 15 byte-pad mapping
mp = {}
for i in range(256):
  one_byte = bytes([i])
  query = cmsc284padbytes(one_byte)
  ctext = make_query('three', 'ruolinzheng', query)
  # len(ctext) is 64, three blocks, need first block
  mp[ctext[:16]] = one_byte

In [47]:
# manipulate len of query to push last byte of FLAG to its own block
ctext = make_query('three', 'ruolinzheng', '')
# len 48, 3 blocks, need last block
mp[ctext[-16:]] # last byte of FLAG

b' '

In [52]:
# second-to-last byte of FLAG
mp = {}
for i in range(256):
  two_bytes = bytes([i]) + b' '
  query = cmsc284padbytes(two_bytes)
  ctext = make_query('three', 'ruolinzheng', query)
  # len(ctext) is 64, three blocks, need first block
  mp[ctext[:16]] = two_bytes

In [58]:
ctext = make_query('three', 'ruolinzheng', bytes(1))
# len 48, 3 blocks, need last block
mp[ctext[-16:]] # last byte of FLAG

b'f '

In [61]:
def get_byte(prev_bytes):
  mp = {}
  for i in range(256):
    query_bytes = bytes([i]) + prev_bytes
    query = cmsc284padbytes(query_bytes)
    ctext = make_query('three', 'ruolinzheng', query)
    # len(ctext) is 64, three blocks, need first block
    mp[ctext[:16]] = query_bytes
  ctext = make_query('three', 'ruolinzheng', bytes(len(prev_bytes)))
  return mp[ctext[-16:]]

In [62]:
get_byte(b'')

b' '

In [63]:
get_byte(b' ')

b'f '

In [64]:
get_byte(b'f ')

b'lf '

In [67]:
# retrieve an entire block first
def get_block():
  prev_bytes = b''
  for i in range(16):
    prev_bytes = get_byte(prev_bytes)
    print(prev_bytes)
  return prev_bytes

In [68]:
get_block()

b' '
b'f '
b'lf '
b'elf '
b'self '
b'yself '
b'myself '
b' myself '
b'l myself '
b'el myself '
b'uel myself '
b'fuel myself '
b' fuel myself '
b'e fuel myself '
b'le fuel myself '


KeyError: b'j\xe1\xb3\x98\xd5\xbd\x91\\\xef\xf3@\xac\x04\xd2\x1a\x8a'